#  Latent semantic Analysis

Pemodelan Topik adalah teknik tanpa pengawasan untuk menemukan tema dari dokumen yang diberikan. Teknik ini mengekstrak sekumpulan kata kunci yang muncul bersama. Kata kunci yang muncul bersama ini mewakili sebuah topik. Sebagai contoh, saham, pasar, ekuitas, reksadana akan mewakili topik 'investasi saham'.

In [1]:
_= !pip install nltk
_= !pip install indoNLP
_= !pip install pydrive

In [2]:
import pandas as pd #pandas
import numpy as np #numpy
import re #regex
import string #string population
from nltk.tokenize import word_tokenize #tokenize
from nltk.corpus import stopwords #stopword
from indoNLP.preprocessing import replace_slang #slank word
from nltk.stem.porter import PorterStemmer #stemming

import os
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth                  
from oauth2client.client import GoogleCredentials

## Credential Twitter

In [4]:
auth.authenticate_user()                         # Follow prompt in the authorization process
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
your_module = drive.CreateFile({"id": "1XuqdhvVPSaAqAXhYYhzFqQJBD3INE6nv"})
your_module.GetContentFile("credential.py") 

In [5]:
from credential import twitterApi

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

## Crawling Twitter

In [ ]:
# Deklarasi kebutuhan modul/library 
import tweepy
from tweepy import OAuthHandler
import pandas as pd
import re 

# Inisialisasi kode akses token dan kode secret akses token
consumer_key    = twitterApi['consumer_key']
consumer_secret = twitterApi['consumer_secret']
access_token    = twitterApi['access_token']
access_token_secret = twitterApi['access_token_secret']


# Membuat objek otentifikasi
authenticate = tweepy.OAuthHandler(consumer_key, consumer_secret) 
    
# Memasukkan kode akses token dan kode secret akses token
authenticate.set_access_token(access_token, access_token_secret) 
    
# Membuat objek API ketika melewatkan informasi otentifikasi
api = tweepy.API(authenticate)


# Mengekstrak 200 twit terakhir dari user Twitter
# posts = api.user_timeline(screen_name="capres2024", count = 200, lang ="en", tweet_mode="extended")
hashtag="capres2024"
limit=300
posts = tweepy.Cursor(api.search_tweets, q=hashtag+' -filter:tweets', tweet_mode="extended", result_type='mixed').items(limit)

# Membuat dataframe dengan kolom bernama Tweets 
df = pd.DataFrame([[tweet.full_text, tweet.user.location, tweet.user.screen_name] for tweet in posts], columns=['Tweets', 'Location', 'Name'])

# Menampilkan 5 twit terakhir yang sudah dibersihkan dalam bentuk dataframe
df

,Tweets,Location,Name
0,Sudah terbukti kan? Kalau @ganjarpranowo merup...,Indonesia,Ganjaran_app
1,"Menjadi kandidat terkuat dalam capres 2024, @g...",Indonesia,Ganjaran_app
2,Anies Baswedan berjanji jika terpilih sebagai ...,Pulo Gadung,tvOneNews
3,RT @Ganjaran_app: Sifatnya yg dekat dgn anak2 ...,,Markowsiego_
4,RT @Ganjaran_app: Sifatnya yg dekat dgn anak2 ...,,BangunIni
...,...,...,...
295,"""Saya kirim kuda ke Sumbar, tapi dengan syarat...",bogor,fauzardiansyah
296,Menteri Pertahanan Prabowo Subianto janji baka...,bogor,fauzardiansyah
297,Prabowo ingin pemda Tanah Datar untuk terus me...,bogor,fauzardiansyah
298,”Saya sering berada di lapangan ini saat kecil...,bogor,fauzardiansyah


## Prepocessing data

In [ ]:
class Prepocessing:
  def __init__(self):
    self.listStopword =  set(stopwords.words('indonesian'))
    self.stemmer = PorterStemmer()

  def remove_emoji(self, string): #remove emoji
    emoji_pattern = re.compile("["
      u"\U0001F600-\U0001F64F"  # emoticons
      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
      u"\U0001F680-\U0001F6FF"  # transport & map symbols
      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
      u"\U00002702-\U000027B0"
      u"\U000024C2-\U0001F251"
      "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' ', string)

  def remove_unwanted(self, document): #clean text
    # remove user mentions
    document = re.sub("@[A-Za-z0-9_]+"," ", document)
    # remove URLS 
    document = re.sub(r'http\S+', ' ', document)
    # remove hashtags
    document = re.sub("#[A-Za-z0-9_]+","", document)
    # remove emoji's
    document = self.remove_emoji(document)
    # remove punctuation
    document = re.sub("[^0-9A-Za-z ]", "" , document)
    # remove double spaces
    document = document.replace('  '," ")
    return document.strip()
  
  def tokenize(self, text): #tokenize -> memisah kalimat 
    return word_tokenize(text.translate(str.maketrans('', '', string.punctuation)).lower())
  
  def stopWord(self, text): #stopword -> menghapus kata hubung
    return [kata for kata in text if kata not in self.listStopword]
  
  def slank_word(self, text): #slank word -> mengganti kata yang tidak baku
    return [replace_slang(kata) for kata in text]

  def stemming(self, text): #stemming -> mengganti kata menjadi kata dasar
    return " ".join([self.stemmer.stem(kata) for kata in text])

In [ ]:
preprocessing = Prepocessing()

### Clean Text

In [ ]:
df['clean'] = df['Tweets'].apply(lambda x: preprocessing.remove_unwanted(x))

### Tokenize

In [ ]:
df['tokenize'] = df['clean'].apply(lambda x: preprocessing.tokenize(x))

### stopword

In [ ]:
df['stopword'] = df['tokenize'].apply(lambda x: preprocessing.stopWord(x))

### Slank Word

In [ ]:
df['slankword'] = df['stopword'].apply(lambda x: preprocessing.slank_word(x))

### Stemming

In [ ]:
df['stem'] = df['slankword'].apply(lambda x: preprocessing.stemming(x))

In [ ]:
df[['Tweets', 'clean', 'tokenize', 'stopword', 'slankword', 'stem']]

,Tweets,clean,tokenize,stopword,slankword,stem
0,Sudah terbukti kan? Kalau @ganjarpranowo merup...,Sudah terbukti kan Kalau merupakan pemimpin y...,"[sudah, terbukti, kan, kalau, merupakan, pemim...","[terbukti, pemimpin, cocok, kalangan, kalangan...","[terbukti, pemimpin, cocok, kalangan, kalangan...",terbukti pemimpin cocok kalangan kalangan anak...
1,"Menjadi kandidat terkuat dalam capres 2024, @g...",Menjadi kandidat terkuat dalam capres 2024 me...,"[menjadi, kandidat, terkuat, dalam, capres, 20...","[kandidat, terkuat, capres, 2024, sinyal, duku...","[kandidat, terkuat, capres, 2024, sinyal, duku...",kandidat terkuat capr 2024 sinyal dukungan
2,Anies Baswedan berjanji jika terpilih sebagai ...,Anies Baswedan berjanji jika terpilih sebagai ...,"[anies, baswedan, berjanji, jika, terpilih, se...","[anies, baswedan, berjanji, terpilih, presiden...","[anies, baswedan, berjanji, terpilih, presiden...",ani baswedan berjanji terpilih presiden menjal...
3,RT @Ganjaran_app: Sifatnya yg dekat dgn anak2 ...,RT Sifatnya yg dekat dgn anak2 membuat menda...,"[rt, sifatnya, yg, dekat, dgn, anak2, membuat,...","[rt, sifatnya, yg, dgn, anak2, mendapatkn, 3, ...","[rt, sifatnya, yang, dengan, anak-anak, mendap...",rt sifatnya yang dengan anak-anak mendapatkan ...
4,RT @Ganjaran_app: Sifatnya yg dekat dgn anak2 ...,RT Sifatnya yg dekat dgn anak2 membuat menda...,"[rt, sifatnya, yg, dekat, dgn, anak2, membuat,...","[rt, sifatnya, yg, dgn, anak2, mendapatkn, 3, ...","[rt, sifatnya, yang, dengan, anak-anak, mendap...",rt sifatnya yang dengan anak-anak mendapatkan ...
...,...,...,...,...,...,...
295,"""Saya kirim kuda ke Sumbar, tapi dengan syarat...",Saya kirim kuda ke Sumbar tapi dengan syarat p...,"[saya, kirim, kuda, ke, sumbar, tapi, dengan, ...","[kirim, kuda, sumbar, syarat, pencinta, binata...","[kirim, kuda, sumbar, syarat, pencinta, binata...",kirim kuda sumbar syarat pencinta binatang dir...
296,Menteri Pertahanan Prabowo Subianto janji baka...,Menteri Pertahanan Prabowo Subianto janji baka...,"[menteri, pertahanan, prabowo, subianto, janji...","[menteri, pertahanan, prabowo, subianto, janji...","[menteri, pertahanan, prabowo, subianto, janji...",menteri pertahanan prabowo subianto janji bibi...
297,Prabowo ingin pemda Tanah Datar untuk terus me...,Prabowo ingin pemda Tanah Datar untuk terus me...,"[prabowo, ingin, pemda, tanah, datar, untuk, t...","[prabowo, pemda, tanah, datar, mengembangkan, ...","[prabowo, pemda, tanah, datar, mengembangkan, ...",prabowo pemda tanah datar mengembangkan olahra...
298,”Saya sering berada di lapangan ini saat kecil...,Saya sering berada di lapangan ini saat kecil ...,"[saya, sering, berada, di, lapangan, ini, saat...","[lapangan, sumbar, kampung, halaman, prabowo, ...","[lapangan, sumbar, kampung, halaman, prabowo, ...",lapangan sumbar kampung halaman prabowo capres...


In [ ]:
df_final = df[['stem']]
df_final

,stem
0,terbukti pemimpin cocok kalangan kalangan anak...
1,kandidat terkuat capr 2024 sinyal dukungan
2,ani baswedan berjanji terpilih presiden menjal...
3,rt sifatnya yang dengan anak-anak mendapatkan ...
4,rt sifatnya yang dengan anak-anak mendapatkan ...
...,...
295,kirim kuda sumbar syarat pencinta binatang dir...
296,menteri pertahanan prabowo subianto janji bibi...
297,prabowo pemda tanah datar mengembangkan olahra...
298,lapangan sumbar kampung halaman prabowo capres...


In [ ]:
df_save = df_final.drop_duplicates().reset_index(drop=True)
df_save

,stem
0,terbukti pemimpin cocok kalangan kalangan anak...
1,kandidat terkuat capr 2024 sinyal dukungan
2,ani baswedan berjanji terpilih presiden menjal...
3,rt sifatnya yang dengan anak-anak mendapatkan ...
4,rt cnbc indonesia mencoba mencari capr popul m...
...,...
157,anak muda joki berbakat kirim jakarta latih bi...
158,peluang cak imin pendamp prabowo kontestasi pi...
159,prabowo berharap olahraga berkuda berkembang p...
160,menteri pertahanan prabowo subianto janji bibi...


## Save dataset

In [ ]:
df_save.to_csv('datasetCapres2024.csv', index=False)

## Modelling data twitter

### load dataset

In [ ]:
df_main = pd.read_csv('https://raw.githubusercontent.com/farisulhaq/dataset/main/datasetCapres2024.csv')
df_main

,stem
0,terbukti pemimpin cocok kalangan kalangan anak...
1,kandidat terkuat capr 2024 sinyal dukungan
2,ani baswedan berjanji terpilih presiden menjal...
3,jaga netralita ciptakan suasana kondusif
4,asn netral pemilu 2024 tercipta pemilu damai
...,...
107,semoga olahraga berkuda berkembang pesat sumba...
108,prabowo pemda tanah datar stakehold sumatera b...
109,prabowo subianro capr 2024 menggemari olahraga...
110,menhan prabowo subianto menghadiri gelaran pac...


### tf-idf

merubah text menjadi feature

In [ ]:
# Vectorize document using TF-IDF
tfidf = TfidfVectorizer()

In [ ]:
documents_list = df_main.values.reshape(-1,).tolist()

In [ ]:
train_data = tfidf.fit_transform(documents_list)  

### Perform SVD


SVD adalah teknik dekomposisi matriks yang memfaktorkan matriks dalam hasil kali matriks. Scikit-learn menawarkan TruncatedSVD untuk melakukan SVD. Mari kita lihat contoh di bawah ini:

In [ ]:
# Define the number of topics or components
num_components=20

# Create SVD object
lsa = TruncatedSVD(n_components=num_components, n_iter=100, random_state=42)

# Fit SVD model on data
lsa.fit_transform(train_data)

# Get Singular values and Components 
Sigma = lsa.singular_values_ 
V_transpose = lsa.components_.T

In [ ]:
# Print the topics with their terms
terms = tfidf.get_feature_names_out() 

for index, component in enumerate(lsa.components_):
    zipped = zip(terms, component)
    top_terms_key=sorted(zipped, key = lambda t: t[1], reverse=True)[:5]
    top_terms_list=list(dict(top_terms_key).keys())
    print("Topic "+str(index)+": ",top_terms_list)

Topic 0:  ['prabowo', 'capres2024', 'dekade08', 'mendingprabowo', 'terusmajubersamaprabowo']
Topic 1:  ['tanah', 'datar', 'kuda', 'pacu', 'open']
Topic 2:  ['berkuda', 'olahraga', 'berkembang', 'pesat', 'sumbar']
Topic 3:  ['partai', 'politik', 'buruh', 'mengaku', 'bergabung']
Topic 4:  ['pemilu', '2024', 'asn', 'jaga', 'netralita']
Topic 5:  ['barat', 'sumatera', 'kasih', 'terima', 'diterima']
Topic 6:  ['capr', 'subianto', 'rakyat', 'minang', 'salah']
Topic 7:  ['capr', 'barat', 'sumatera', 'ani', 'kasih']
Topic 8:  ['erick', 'nu', 'thohir', 'alat', 'kader']
Topic 9:  ['jokowi', 'diuntungkan', 'kepuasan', 'presiden', 'tingkat']
Topic 10:  ['ningrat', 'sosok', 'disorot', 'jejak', 'kariernya']
Topic 11:  ['rt', 'jaga', 'netralita', 'elektabilita', 'rakyat']
Topic 12:  ['ani', 'minang', 'jokowi', 'setan', 'baswedan']
Topic 13:  ['sumbar', 'kampung', 'lapangan', 'halaman', 'ani']
Topic 14:  ['gerindra', 'kader', 'calon', 'ketua', 'salah']
Topic 15:  ['karier', '2004', 'golkar', 'bergabun